In [5]:
import asyncio
import json
import re
from google.adk.runners import Runner
from google.genai import types
from google.adk.sessions import InMemorySessionService
from dotenv import load_dotenv

# Import the fully assembled root agent from our coordinator module
from agents.coordinator import root_agent

# Use a shared session service for the application
session_service = InMemorySessionService()

from tools.code_analysis_tools import analyze_code_structure

In [4]:
# 1. Load the source code we want to test
try:
    with open("sample_code.py", "r") as f:
        source_code_to_test = f.read()
except FileNotFoundError:
    print("Error: `sample_code.py` not found. Please ensure the file exists.")
    

In [6]:
# 2. Instantiate the ADK Runner with our master agent
# We pass the shared session_service instance here.
runner = Runner(
    app_name="autotest_suite_generator",
    agent=root_agent,
    session_service=session_service
)

In [7]:
# 3. Create a session for this run
session = await runner.session_service.create_session(
    app_name="autotest_suite_generator",
    user_id="end_user"
)

In [8]:
# 4. Format the initial user request as a JSON object
# The `initialize_state` callback on our root agent will parse this.
initial_request = json.dumps({
    "source_code": source_code_to_test,
    "language": "python"
})

print(f"\n[USER REQUEST] Generating tests for:\n---\n{source_code_to_test}\n---\n")

user_message = types.Content(
    role="user",
    parts=[types.Part(text=initial_request)]
)


[USER REQUEST] Generating tests for:
---
"""A sample module for demonstration."""

class Calculator:
    """A simple calculator class."""
    
    def add(self, a: int, b: int) -> int:
        """Adds two numbers together."""
        return a + b

def greet(name: str) -> str:
    """Returns a greeting message."""
    return f"Hello, {name}"
---



In [9]:
analyze_code_structure(source_code_to_test, 'python')

{'status': 'success',
 'structure': [{'type': 'class',
   'name': 'Calculator',
   'docstring': 'A simple calculator class.',
   'methods': [{'name': 'add',
     'docstring': 'Adds two numbers together.',
     'parameters': [{'name': 'self', 'annotation': None},
      {'name': 'a', 'annotation': 'int'},
      {'name': 'b', 'annotation': 'int'}],
     'return_type': 'int'}]},
  {'name': 'greet',
   'docstring': 'Returns a greeting message.',
   'parameters': [{'name': 'name', 'annotation': 'str'}],
   'return_type': 'str',
   'type': 'function'}]}